In [4]:
import pandas as pd
from collections import Counter, defaultdict
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

def clean(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    words = text.split()
    return [w for w in words if w not in stop_words]

# Load data
df = pd.read_json("../data/experiment.jsonl", lines=True)

# Collect word counts by race
word_counts = defaultdict(Counter)
total_words = Counter()

for _, row in df.iterrows():
    race = row["race"]
    words = clean(row["response"])
    word_counts[race].update(words)
    total_words.update(words)

# Combine other races for comparison
race_terms = {
    "black", "white", "asian", "hispanic", "native", "american",
    "pacific", "islander", "hawaiian", "indianalaska", "african",
    "hawaiianpacific", "nativeamerican", "caucasian", "americanalaska", "alaska"
}

def log_odds_ratio_filtered(target_race, word_counts, alpha=0.01, top_n=20):
    all_words = set(w for c in word_counts.values() for w in c)
    target_counts = word_counts[target_race]
    other_counts = Counter()
    for race, counts in word_counts.items():
        if race != target_race:
            other_counts.update(counts)

    results = []
    for word in all_words:
        if word in race_terms:
            continue  # skip racial terms

        a = target_counts[word] + alpha
        b = sum(target_counts.values()) + alpha * len(all_words)
        c = other_counts[word] + alpha
        d = sum(other_counts.values()) + alpha * len(all_words)

        log_odds = np.log((a / (b - a)) / (c / (d - c)))

        if log_odds > 0:  # only keep positive distinctive words
            results.append((word, log_odds, target_counts[word]))

    return sorted(results, key=lambda x: -x[1])[:top_n]  # top-N only

for race in word_counts:
    print(f"\nWords most overrepresented in {race} (excluding race terms):")
    for word, logodds, count in log_odds_ratio_filtered(race, word_counts):
        print(f"{word:<15} log-odds: {logodds:.3f}  count: {count}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Words most overrepresented in Black (excluding race terms):
coiled          log-odds: 9.734  z: 0.97  p: 0.3304  count: 33
closelycropped  log-odds: 9.734  z: 0.97  p: 0.3304  count: 33
curled          log-odds: 8.539  z: 0.85  p: 0.3934  count: 10
wellmaintained  log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
sped            log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
ongoing         log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
concerns        log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
shortcropped    log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
confidence      log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
jumbled         log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
concealment     log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
inch            log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
deep            log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
panic           log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
stepping        log-odds: 6.933  z: 0.69  p: 0.4892  count: 2
violen

In [1]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

# Clean and tokenize function
def clean(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    words = text.split()
    return [w for w in words if w not in stop_words]

# Load data
df = pd.read_json("data/experiment.jsonl", lines=True)

# Count words by (race, category)
group_word_counts = defaultdict(Counter)
total_words = Counter()

for _, row in df.iterrows():
    key = (row["race"], row["category"])
    words = clean(row["response"])
    group_word_counts[key].update(words)
    total_words.update(words)

# Race terms to exclude from log-odds analysis
race_terms = {
    "black", "white", "asian", "hispanic", "native", "american",
    "pacific", "islander", "hawaiian", "indianalaska", "african",
    "hawaiianpacific", "nativeamerican", "caucasian", "americanalaska", "alaska"
}

# Log-odds ratio calculation across (race, category) groups
def log_odds_by_group(target_key, word_counts, alpha=0.01, top_n=15):
    all_words = set(w for c in word_counts.values() for w in c)
    target_counts = word_counts[target_key]
    other_counts = Counter()
    for group_key, counts in word_counts.items():
        if group_key != target_key:
            other_counts.update(counts)

    results = []
    for word in all_words:
        if word in race_terms:
            continue

        a = target_counts[word] + alpha
        b = sum(target_counts.values()) + alpha * len(all_words)
        c = other_counts[word] + alpha
        d = sum(other_counts.values()) + alpha * len(all_words)

        log_odds = np.log((a / (b - a)) / (c / (d - c)))

        if log_odds > 0:
            results.append((word, log_odds, target_counts[word]))

    return sorted(results, key=lambda x: -x[1])[:top_n]

# Run for each (race, category)
for group_key in group_word_counts:
    race, category = group_key
    print(f"\n Most distinctive words for RACE: {race} | CATEGORY: {category}")
    for word, logodds, count in log_odds_by_group(group_key, group_word_counts):
        print(f"{word:<15} log-odds: {logodds:.3f}  count: {count}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



 Most distinctive words for RACE: Black | CATEGORY: Violent Offense
mediumdark      log-odds: 8.161  count: 2
deep            log-odds: 8.161  count: 2
inch            log-odds: 8.161  count: 2
confidence      log-odds: 8.161  count: 2
ongoing         log-odds: 8.161  count: 2
seriousness     log-odds: 7.473  count: 1
conditions      log-odds: 7.473  count: 1
younger         log-odds: 7.473  count: 1
jostle          log-odds: 7.473  count: 1
hedisplayed     log-odds: 7.473  count: 1
traveling       log-odds: 7.473  count: 1
committed       log-odds: 7.473  count: 1
identify        log-odds: 7.473  count: 1
shift           log-odds: 7.473  count: 1
aims            log-odds: 7.473  count: 1

 Most distinctive words for RACE: White | CATEGORY: Violent Offense
lowering        log-odds: 8.140  count: 2
demand          log-odds: 8.140  count: 2
hazel           log-odds: 8.140  count: 2
stab            log-odds: 7.451  count: 1
upward          log-odds: 7.451  count: 1
lip             log-od

In [2]:
# Lexical Analysis for Baseline

import pandas as pd
from collections import Counter
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

# Load baseline dataset
df = pd.read_json("data/baseline.jsonl", lines=True)

# Basic tokenizer
def clean(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)  # remove punctuation
    words = text.split()
    return [w for w in words if w not in stop_words]

# 1. Top 20 most common words overall
all_words = []
for text in df["response"]:
    all_words.extend(clean(text))

overall_counter = Counter(all_words)
print("\n Top 20 Most Frequent Words Overall:")
for word, count in overall_counter.most_common(20):
    print(f"{word:<15} {count}")

# 2. Top 15 words per category
print("\n Top 15 Most Frequent Words by Category:")
for category in df["category"].unique():
    cat_words = []
    for text in df[df["category"] == category]["response"]:
        cat_words.extend(clean(text))
    cat_counter = Counter(cat_words)
    print(f"\n🟦 Category: {category}")
    for word, count in cat_counter.most_common(15):
        print(f"{word:<15} {count}")


 Top 20 Most Frequent Words Overall:
suspect         557
dark            447
black           260
approximately   255
hair            244
build           239
jeans           239
male            235
wearing         234
short           233
tall            223
brown           222
medium          214
appeared        209
hoodie          188
sneakers        182
around          159
appears         148
incident        144
description     135

 Top 15 Most Frequent Words by Category:

🟦 Category: Violent Offense
suspect         204
dark            157
black           106
approximately   82
build           80
jeans           80
medium          79
wearing         79
short           77
hair            77
male            76
tall            76
incident        76
appeared        73
brown           70

🟦 Category: Property Offense
suspect         197
dark            153
black           102
approximately   85
wearing         85
hair            83
short           82
jeans           80
male            79

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
